In [21]:
import torchattacks
import torch
import numpy as np
import json
import logging
from tqdm import trange

import os
import sys
root = os.path.abspath(os.curdir)
sys.path.append(root)

from abs_models import models as mz        # model zoo
from abs_models import utils as u

from utils.models import load_net
from utils.helpers import *
from utils.attack import attack

In [2]:
# device = 'cuda:0'
# bs=100
# logging.basicConfig(filename=os.path.join(root,'abs_attacks.txt'), 
#                     level=logging.CRITICAL, 
#                     format="%(message)s")

# logging.critical('Starting evaluation of abs model')
# logging.critical('-----------------------------------------------')

# model = mz.get_VAE(n_iter=50).to(device)              # ABS do n_iter=1 for speedup (but ess accurate)
# x, y_target = u.get_batch(bs)  
# logits = model(u.n2t(x))             # returns torch.tensor, shape (batch_size, n_channels, nx, ny)

# y = np.argmax(logits.cpu(), axis=1)
# acc = 100*float(np.sum(np.array(y)==y_target)/bs)
# print(acc)
# logging.critical('clean accuracy: %.2f'%acc)

In [22]:
mnist_path = '/home/mbeliaev/home/code/robust-l0/new_trained/final_500/MNIST/1/cnn_small/simple/k_25/adv_25/0/setup.json'
cifar_path = '/home/mbeliaev/home/code/robust-l0/new_trained/final_500/CIFAR/1/VGG16/simple/k_50/adv_50/0/setup.json'

In [52]:
device = 'cuda:0'
bs = 100
cfg_path = cifar_path

config = json.load(open(cfg_path,'rb'))
data = prep_data(root, bs=bs, dataset=config['dataset'])
result_path = cfg_path[0:-len('/setup.json')]

model = load_net(result_path, device, input_shape=data['x_test'][0].shape)
model.to(device)

Files already downloaded and verified
Files already downloaded and verified


Net(
  (trunc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): trunc_simple()
    (2): Unflatten(dim=1, unflattened_size=torch.Size([3, 32, 32]))
  )
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, trac

In [53]:
atk = torchattacks.OnePixel(model,pixels=1)
# atk = torchattacks.SparseFool(model)

# atk = torchattacks.LGV(model)
x,y  = data['x_test'][0], data['y_test'][0]
adv_x = atk(torch.tensor(x),torch.tensor(y))

y_adv = np.argmax(model(adv_x).detach().cpu(), axis=1)
r_acc = 100*float(np.sum(np.array(y_adv)==y)/bs)
print('Robust Accuracy: ',r_acc,'%')


# x.shape
# adv_x = atk(torch.tensor(x),torch.tensor(y_target))           

/tmp/ipykernel_2780951/207749188.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adv_x = atk(torch.tensor(x),torch.tensor(y))
/home/mbeliaev/miniconda3/envs/robust/lib/python3.8/site-packages/torchattacks/attacks/_differential_evolution.py:592: RuntimeWarning: divide by zero encountered in double_scalars
  convergence=self.tol / convergence) is True):


Robust Accuracy:  0.0 %


In [54]:
(adv_x.detach().cpu() != x).sum()//100

tensor(3)

In [16]:
# atks to use: OnePixel SparseFool Pixle LGV
device = 'cuda:0'
bs=4
model = mz.get_VAE(n_iter=2).to(device)              # ABS do n_iter=1 for speedup (but ess accurate)
x, y_target = u.get_batch(bs)  

atk = torchattacks.OnePixel(model,pixels=1)
adv_x = atk(torch.tensor(x),torch.tensor(y_target))

ABS model
model loaded
setting random seed
done creating samples


In [17]:
# compute clean acc
logits = model(u.n2t(x))
y = np.argmax(logits.cpu(), axis=1)
acc = 100*float(np.sum(np.array(y)==y_target)/bs)
print('Clean Accuracy: ',acc,'%')

logits_adv = model(adv_x)
y_adv = np.argmax(logits_adv.cpu(), axis=1)
r_acc = 100*float(np.sum(np.array(y_adv)==y_target)/bs)
print('Robust Accuracy: ',r_acc,'%')

Clean Accuracy:  96.0 %
Robust Accuracy:  92.0 %
